In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import pipeline
import pandas as pd
import numpy as np
from ganacheconnector import contractConnector
import json
from web3 import Web3, HTTPProvider

In [61]:
from datetime import date, datetime

from dateutil.relativedelta import relativedelta

start_date="2020-12-11"
duration = 3
end_date = datetime.strptime(start_date, "%Y-%m-%d") + relativedelta(months=+duration)
end_date = end_date.strftime('%Y-%m-%d')
end_date

'2021-03-11'

In [2]:
blockchain_address = 'http://127.0.0.1:8545'
compiled_contract_path = '../build/contracts/P2PTest.json'
deployed_contract_address = '0x6d72dC5be82840419bde6758D04Cc37102eB0473'
defaultAccount = 0

web3 = Web3(HTTPProvider(blockchain_address))
web3.eth.accounts[defaultAccount]
web3.eth.defaultAccount = web3.eth.accounts[defaultAccount]
c = contractConnector(blockchain_address,compiled_contract_path,deployed_contract_address,defaultAccount)

In [30]:
#classifier = pipeline("zero-shot-classification",model="facebook/bart-large-mnli")
classifier = pipeline("zero-shot-classification",model="valhalla/distilbart-mnli-12-3")
categories = ['grocery', 'medical bills', 'home improvement', 'debt', 'Job Loss', 'auto repair', 'pet care', 'funeral', 'business needs']

Downloading:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [31]:
df_all_loans = c.get_all_loan_struct_info()
df_all_loans.head()

,shortDescription,loanDescription,loanId,loanDuration,totalAmount,amountFunded,interestRate,numFunders,EXISTS,fundedStatus
0,Grocery help needed,I have not yet received my paycheck and I need...,0,1,120,95,3,3,True,False
0,Kitched home rennovation,I have just moved into the neighborhood and am...,1,12,1000,222,10,1,True,False
0,Credit card payment help,The next billing cycle for my credit card debt...,2,2,375,100,4,1,True,False
0,"Lost job, need help with bills",I was a cleaning staff at the local shopping m...,3,3,775,75,5,1,True,False
0,Unforseen accident and car repair,I got into a car accident last month and my ca...,4,5,1250,500,3,1,True,False


In [44]:
def get_top_tag(classifier,loanDescription):
    result = classifier(loanDescription,categories,multi_label=False)
    top_tag = result["labels"][0]
    return top_tag

In [49]:
query = df_all_loans.loanDescription.tolist()[3]
get_top_tag(classifier,query)

'Job Loss'

In [32]:
results = classifier(df_all_loans.loanDescription.tolist(),categories,multi_label=False)

In [33]:
l = []
for i in results:
    j = [i["sequence"],i["labels"][0],i["scores"][0]]
    l.append(j)
df_tags = pd.DataFrame(l,columns=["loanDescription", "TopTag","Score"])
df_tags

,loanDescription,TopTag,Score
0,I have not yet received my paycheck and I need...,grocery,0.853781
1,I have just moved into the neighborhood and am...,home improvement,0.829831
2,The next billing cycle for my credit card debt...,debt,0.882973
3,I was a cleaning staff at the local shopping m...,Job Loss,0.597745
4,I got into a car accident last month and my ca...,auto repair,0.434784
5,I had to take my son to the emergency room aft...,medical bills,0.496264
6,My grandma lives in an old house in Georgia by...,home improvement,0.864416
7,We found a blocked pipe in the house. The esti...,home improvement,0.578507
8,I just started my own little coffee shop in th...,business needs,0.776404
9,need lunch for the week,grocery,0.403252


In [41]:
query = "I am looking to help someone pay some hospital bills from an accident they incurred."
result = classifier(query,categories,multi_label=False)
top_tag = result["labels"][0]

In [15]:
results

[{'sequence': 'I have just moved into the neighborhood and am planning to rennovate my kitchen. Currently do not have enough funds to pay the job in full.',
  'labels': ['home improvement',
   'debt',
   'Job Loss',
   'business needs',
   'medical bills',
   'charity',
   'pet care',
   'grocery',
   'funeral',
   'auto repair'],
  'scores': [0.7746440768241882,
   0.15895119309425354,
   0.04492936283349991,
   0.011712076142430305,
   0.0018375774379819632,
   0.001818630495108664,
   0.0017931407783180475,
   0.0016324493335559964,
   0.0014066359726712108,
   0.0012748903827741742]},
 {'sequence': 'The next billing cycle for my credit card debt starts in 3 days. Although I have been diligently paying part of it off every month, the snowball gets bigger and bigger. I really don’t want to pay the high interest rate anymore. It will be helpful if I can get a small loan to pay it all off, then my monthly payment will be much more affordable.',
  'labels': ['debt',
   'Job Loss',
   'b

In [ ]:
def analysis(query, esg_cat):
    result = model(query, esg_cat, multi_class=True)
    df_result = pd.DataFrame(list(zip(result["labels"],result["scores"])),columns=['Topic','Score'])
    top_cats = df_result[df_result['Score']>=0.8]['Topic'].tolist()
    print("\n ",query)
    plotter(df_result)
    return top_cats